# API Client
The following tutorial demonstrates how to access the PKDB API with python.

For more information and installation information see:  
https://github.com/matthiaskoenig/pkdb

In [1]:
URL_PKDB = "http://0.0.0.0:8000/"

In [2]:
# import libraries
import coreapi
import requests
import json
from pprint import pprint
from requests.auth import HTTPBasicAuth

In [3]:
#response = requests.get('http://0.0.0.0:8000/', auth=HTTPBasicAuth('janek', 'test'))

In [4]:
#r = requests.post("http://0.0.0.0:8000/api-auth/login/?next=/api/v1/", data={'username': 'janek', 'password': 'test'})
#print(r.status_code, r.reason)
#r.json()["token"]

## Create client & API endpoint

In [16]:
#auth = coreapi.auth.BasicAuthentication(
#username="janek",
#password="test", )
client = coreapi.Client()

In [17]:
# get the api scema
document = client.get(URL_PKDB)
print(document.title)
print(document.url)

PkBD API
http://0.0.0.0:8000/


In [18]:
# overview available methods
print(document)

<PkBD API "http://0.0.0.0:8000/">
    authors: {
        list([page], [first_name], [last_name], [search])
        create([first_name], [last_name])
        read(id, [first_name], [last_name], [search])
        update(id, [first_name], [last_name], [first_name], [last_name], [search])
        partial_update(id, [first_name], [last_name], [first_name], [last_name], [search])
        delete(id, [first_name], [last_name], [search])
    }
    references: {
        list([page], [pmid], [doi], [title], [abstract], [journal], [date], [authors], [search])
        create(sid, title, date, authors, [pmid], [doi], [abstract], [journal], [pdf])
        read(id, [pmid], [doi], [title], [abstract], [journal], [date], [authors], [search])
        update(id, sid, title, date, authors, [pmid], [doi], [abstract], [journal], [pdf], [pmid], [doi], [title], [abstract], [journal], [date], [authors], [search])
        partial_update(id, [sid], [pmid], [doi], [title], [abstract], [journal], [date], [authors],

## Example queries
Show all authors

In [19]:
data = client.action(document,["authors", "list"])
print(json.dumps(data, indent=2))

{
  "count": 1,
  "next": null,
  "previous": null,
  "results": [
    {
      "id": 1,
      "first_name": "Lucek",
      "last_name": "Grzegorzewski"
    }
  ]
}


Create author

In [20]:
data = client.action(document,["authors", "create"], params={'first_name':'Lucek', 'last_name':'Grzegorzewski'})
print(data)

OrderedDict([('id', 1), ('first_name', 'Lucek'), ('last_name', 'Grzegorzewski')])


In [21]:
# Filter for authors first name = Lucek
data = client.action(document, ["authors", "list"], params={"first_name": "Lucek"} )
print(json.dumps(data, indent=2))

{
  "count": 1,
  "next": null,
  "previous": null,
  "results": [
    {
      "id": 1,
      "first_name": "Lucek",
      "last_name": "Grzegorzewski"
    }
  ]
}


In [22]:
# Search for author
data = client.action(document, ["authors", "list"], params={"search": "Gr"} )
print(json.dumps(data, indent=2))

{
  "count": 1,
  "next": null,
  "previous": null,
  "results": [
    {
      "id": 1,
      "first_name": "Lucek",
      "last_name": "Grzegorzewski"
    }
  ]
}


In [31]:
# Create study
study_dict = {
    "pmid": 16198659,
    "sid": "Granfors2005",
    "date": "2005-11-02",
    "journal": "Clinical pharmacology and therapeutics",
    "title": "Oral contraceptives containing ethinyl estradiol and gestodene markedly increase plasma concentrations and effects of tizanidine by inhibiting cytochrome P450 1A2.",
    "abstract": "OCs containing ethinyl estradiol and gestodene increase, to a clinically significant extent, the plasma concentrations and effects of tizanidine, probably mainly by inhibiting its CYP1A2-mediated presystemic metabolism. Care should be exercised when tizanidine is prescribed to OC users.",
    "groups":,
    "authors": [
        {
            "first_name": "Marika T",
            "last_name": "Granfors"
        },
        {
            "first_name": "Janne T",
            "last_name": "Backman"
        },
        {
            "first_name": "Jouko",
            "last_name": "Laitila"
        },
        {
            "first_name": "Pertti J",
            "last_name": "Neuvonen"
        }
    ],
    "doi": ""
}
client.action(document,["references", "create"], params=study_dict)

ParameterError: {'groups': 'Unknown parameter.'}

In [32]:
import pandas as pd

In [48]:
path ="/home/janekg89/Develop/Pycharm_Projects/pkdb/pkdb_app/data/caffeine/Subjects.tsv"

In [53]:
def subjects_load(json_reference):
    data_pd = pd.read_csv(path,delimiter='\t')
    this_data = data_pd[data_pd["study"] == json_reference["json"]["sid"]]
    json = {**json_reference["json"]}
    json["groups"] = []
    print(len(this_data))
    for name, row in this_data.iterrows():
        yield {"json": json, "group": row.to_dict()}# "reference_path": json_reference["reference_path"]}

In [54]:
data = {"json" : study_dict }
this = subjects_load(data)

In [55]:
next(this)

2


{'json': {'pmid': 16198659,
  'sid': 'Granfors2005',
  'date': '2005-11-02',
  'journal': 'Clinical pharmacology and therapeutics',
  'title': 'Oral contraceptives containing ethinyl estradiol and gestodene markedly increase plasma concentrations and effects of tizanidine by inhibiting cytochrome P450 1A2.',
  'abstract': 'OCs containing ethinyl estradiol and gestodene increase, to a clinically significant extent, the plasma concentrations and effects of tizanidine, probably mainly by inhibiting its CYP1A2-mediated presystemic metabolism. Care should be exercised when tizanidine is prescribed to OC users.',
  'groups': [],
  'authors': [{'first_name': 'Marika T', 'last_name': 'Granfors'},
   {'first_name': 'Janne T', 'last_name': 'Backman'},
   {'first_name': 'Jouko', 'last_name': 'Laitila'},
   {'first_name': 'Pertti J', 'last_name': 'Neuvonen'}],
  'doi': ''},
 'group': {'study': 'Granfors2005',
  'subjects': 'S1',
  'keywords': 'control',
  'species': 'homo sapiens',
  'strain': '-'